In [1]:
import gensim.downloader
import numpy as np
from algorithms.RandomProjectionsLsh import RandomProjectionsLsh
from algorithms.PcaBasedLsh import PcaBasedLsh
from algorithms.MinkowskiLsh import L1Lsh, L2Lsh
from algorithms.PcaMinkowskiLsh import PcaL1Lsh, PcaL2Lsh
from algorithms.SuperBitLsh import SuperBitLsh
from algorithms.HierarchicalLsh import HierarchicalLsh
from algorithms.HierarchicalHyperplaneLsh import HierarchicalHyperplaneLsh

# Load vocabulary

In [2]:
import urllib.request


def load_common_words():
    print('Loading words...')
    words = set()
    target_url = 'https://raw.githubusercontent.com/pkLazer/password_rank/master/4000-most-common-english-words-csv.csv'
    is_header_line = True
    for line in urllib.request.urlopen(target_url):
        if is_header_line:
            is_header_line = False
            continue
        words.add(line.decode('utf-8').strip())
    print(f'Words loaded: {len(words)}')

    return words

vocabulary = load_common_words()

Loading words...
Words loaded: 4319


# Embedding Models

Note that first downloads will be very slow, then first memory load will still be slow, but then each model is cached.

In [3]:
class EmbeddingsModel:
    _instances = {}

    @classmethod
    def get(cls, model_name):
        if not model_name in cls._instances:
            cls._instances[model_name] = cls(model_name)
        return cls._instances[model_name]

    def __init__(self, model_name):
        print(f'Loading {model_name} embeddings model')
        self.embeddings_model = gensim.downloader.load(model_name)

    def embed_word(self, word):
        return self.embeddings_model[word]
    
    def embed_words(self, words):
        existing_words = [word for word in words if self.word_exists(word)]
        return np.array(self.embeddings_model.vectors_for_all(existing_words).vectors)
    
    def embed_words_as_dict(self, words):
        existing_words = [word for word in words if self.word_exists(word)]
        embeddings_for_words = self.embeddings_model.vectors_for_all(existing_words).vectors
        return {word: embeddings_for_words[i] for i, word in enumerate(existing_words)}
    
    def word_exists(self, word):
        return word in self.embeddings_model.key_to_index

In [4]:
possible_models = list(gensim.downloader.info()['models'].keys())
print(f'All possible models: {possible_models}')

EMBEDDING_MODEL = 'glove-wiki-gigaword-100'

All possible models: ['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


# Comparisons

In [5]:
def _hash_vocabulary(vocabulary, embeddings_model_name, hasher):
    embeddings_model = EmbeddingsModel.get(embeddings_model_name)
    print(f'Embedding vocabulary of {len(vocabulary)} words')
    vocabulary_embeddings = embeddings_model.embed_words_as_dict(vocabulary)
    print(f'Training hasher with all {len(vocabulary_embeddings)} vocabulary embeddings')
    hasher.fit(np.array(list(vocabulary_embeddings.values())))
    print(f'Hashing all {len(vocabulary_embeddings)} vocabulary embeddings')
    vocabulary_hashes = {word: hasher.hash_vector(embedding) for word, embedding in vocabulary_embeddings.items() }
    return vocabulary_hashes

def _get_hash_words(hash, vocabulary_hashes):
    words = []
    for word, word_hash in vocabulary_hashes.items():
        if word_hash == hash:
            words.append(word)
    return words

def _print_word_neighbourhood(word, vocabulary_hashes):
    hash = vocabulary_hashes[word]
    neighbourhood = _get_hash_words(hash, vocabulary_hashes)
    print(f'Words for word {word}, hash {hash}: {neighbourhood}')

def compare_vocabulary_hashes(embeddings_model_name, hasher, search_terms):
    print(f'Comparing vocabulary hashes for {embeddings_model_name} embeddings model with {hasher.to_string()}')
    vocabulary_hashes = _hash_vocabulary(vocabulary, embeddings_model_name, hasher)
    for term in search_terms:
        _print_word_neighbourhood(term, vocabulary_hashes)

In [6]:
search_terms = ['apple', 'orange', 'banana', 'grape', 'fruit', 'cheese', 'plane', 'train', 'universe', 'love', 'anger']

In [7]:
compare_vocabulary_hashes(EMBEDDING_MODEL, RandomProjectionsLsh(hash_length=11), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with RandomProjectionsLsh (hash_length=11)
Loading glove-wiki-gigaword-100 embeddings model
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 1717: ['apple', 'programming']
Words for word orange, hash 708: ['hazard', 'fever', 'orange', 'brake', 'pink', 'cloud', 'headache', 'shock']
Words for word banana, hash 1007: ['pasta', 'tooth', 'shell', 'vertical', 'snake', 'chop', 'banana', 'operation']
Words for word grape, hash 750: ['nest', 'grape', 'rat', 'cup', 'additional', 'brush']
Words for word fruit, hash 646: ['bacteria', 'laboratory', 'sleep', 'fruit', 'cancer', 'supply']
Words for word cheese, hash 1773: ['cheese']
Words for word plane, hash 983: ['scream', 'killer', 'off', 'out', 'plane']
Words for word train, hash 963: ['heaven', 'train', 'hit', 'blade', 'spot']
Words for word universe, hash 2038: ['battle',

In [8]:
compare_vocabulary_hashes(EMBEDDING_MODEL, PcaBasedLsh(hash_length=14), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with PcaBasedLsh (hash_length=14)
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 10352: ['buy', 'apple', 'energy', 'giant']
Words for word orange, hash 2393: ['leaf', 'orange', 'fur']
Words for word banana, hash 2136: ['bean', 'chicken', 'meat', 'egg', 'shrimp', 'fruit', 'banana', 'pie', 'sugar']
Words for word grape, hash 3160: ['grape']
Words for word fruit, hash 2136: ['bean', 'chicken', 'meat', 'egg', 'shrimp', 'fruit', 'banana', 'pie', 'sugar']
Words for word cheese, hash 2138: ['pour', 'cheese', 'milk', 'vegetable', 'soup', 'pasta', 'chocolate', 'tablespoon', 'mixture', 'tomato', 'flour', 'potato', 'onion', 'dessert', 'sauce', 'juice', 'lemon', 'salad', 'dried', 'honey', 'peanut', 'pepper', 'butter', 'salt', 'garlic', 'corn']
Words for word plane, hash 10672: ['rocket', 'missile', 'radar', 'cargo', 'shel

In [9]:
compare_vocabulary_hashes(EMBEDDING_MODEL, L1Lsh(dimensions_count=100, bucket_width=0.01, count_hash_tables=200), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with L1Lsh (dimensions_count=100, bucket_width=0.01, count_hash_tables=200
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 1533: ['jungle', 'apple', 'step', 'silence', 'definition']
Words for word orange, hash 1225: ['crazy', 'assess', 'astronomer', 'orange', 'processing', 'voter', 'specify']
Words for word banana, hash 1417: ['directly', 'strong', 'proportion', 'climb', 'dissolve', 'banana', 'fierce']
Words for word grape, hash 1722: ['law', 'grape']
Words for word fruit, hash 1614: ['extend', 'tragic', 'fruit']
Words for word cheese, hash 1394: ['cheese', 'finding', 'donor', 'husband', 'practice', 'hunger', 'asleep', 'toilet']
Words for word plane, hash 1690: ['boss', 'try', 'plane']
Words for word train, hash 1229: ['skirt', 'roof', 'suspicious', 'opposite', 'train', 'pizza', 'bathroom', 'commercial', 'puzzl

In [10]:
compare_vocabulary_hashes(EMBEDDING_MODEL, L2Lsh(dimensions_count=100, bucket_width=0.01, count_hash_tables=200), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with L2Lsh (dimensions_count=100, bucket_width=0.01, count_hash_tables=200
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 314: ['margin', 'apple', 'careful', 'there', 'task']
Words for word orange, hash 736: ['direct', 'orange', 'special', 'accountability']
Words for word banana, hash 834: ['poster', 'banana', 'very']
Words for word grape, hash 1808: ['grape']
Words for word fruit, hash 1325: ['image', 'awareness', 'fruit', 'chamber']
Words for word cheese, hash 1159: ['cheese', 'percentage', 'empty']
Words for word plane, hash 1864: ['plane']
Words for word train, hash 741: ['anywhere', 'hidden', 'train', 'supportive', 'occasional', 'increase', 'visual', 'yours', 'fewer', 'courage', 'grip']
Words for word universe, hash 578: ['universe', 'the', 'artifact', 'suspicion']
Words for word love, hash 672: ['treasur

In [11]:
compare_vocabulary_hashes(EMBEDDING_MODEL, PcaL1Lsh(bucket_width=0.005, count_hash_tables=50), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with PcaL1Lsh (bucket_width=0.005, count_hash_tables=50)
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 376: ['instead', 'existing', 'apple', 'lots', 'reserve', 'vote', 'root', 'arrange', 'enact', 'workout', 'craft', 'graduation']
Words for word orange, hash 416: ['writer', 'scheme', 'order', 'orange', 'time', 'alternative', 'discover', 'refer', 'thus', 'household', 'radical', 'blend', 'someone', 'while', 'defend']
Words for word banana, hash 405: ['significance', 'link', 'walking', 'introduce', 'banana', 'upon']
Words for word grape, hash 386: ['judgment', 'below', 'grape', 'lobby', 'compound', 'acknowledge', 'daughter', 'season', 'rape', 'editor', 'pilot']
Words for word fruit, hash 529: ['contribution', 'the', 'sustainable', 'fruit', 'pepper', 'territory', 'fuel']
Words for word cheese, hash 591: ['cheese',

In [12]:
compare_vocabulary_hashes(EMBEDDING_MODEL, PcaL2Lsh(bucket_width=0.005, count_hash_tables=50), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with PcaL2Lsh (bucket_width=0.005, count_hash_tables=50)
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 376: ['instead', 'duty', 'apple', 'lots', 'exact', 'full', 'root', 'arrange', 'enact', 'workout', 'craft', 'graduation']
Words for word orange, hash 416: ['being', 'writer', 'maintenance', 'scheme', 'professional', 'orange', 'time', 'alternative', 'discover', 'refer', 'thus', 'household', 'radical', 'blend', 'someone', 'while', 'defend', 'now']
Words for word banana, hash 405: ['achievement', 'land', 'animal', 'stomach', 'goal', 'significance', 'essay', 'link', 'walking', 'introduce', 'brake', 'considerable', 'banana', 'cord', 'upon']
Words for word grape, hash 386: ['judgment', 'below', 'connection', 'offensive', 'grape', 'lobby', 'compound', 'acknowledge', 'daughter', 'season', 'colorful', 'editor', 'pilot

In [13]:
compare_vocabulary_hashes(EMBEDDING_MODEL, SuperBitLsh(hash_length=14, num_bits_per_batch=4), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with SuperBitLsh(hash_length=14, num_bits_per_batch=4)
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Hashing all 4253 vocabulary embeddings


c:\Projects\LocalitySensitiveHashingAlgorithms\.venv\lib\site-packages\numpy\linalg\linalg.py:2582: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
c:\Projects\LocalitySensitiveHashingAlgorithms\algorithms\SuperBitLsh.py:20: RuntimeWarning: overflow encountered in multiply
  proj = np.dot(vectors[j], vectors[i]) * vectors[j]
c:\Projects\LocalitySensitiveHashingAlgorithms\algorithms\SuperBitLsh.py:24: RuntimeWarning: invalid value encountered in divide
  vectors = vectors / norms
c:\Projects\LocalitySensitiveHashingAlgorithms\algorithms\SuperBitLsh.py:21: RuntimeWarning: invalid value encountered in subtract
  vectors[i] -= proj
c:\Projects\LocalitySensitiveHashingAlgorithms\.venv\lib\site-packages\numpy\linalg\linalg.py:2583: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


Words for word apple, hash 8224: ['bite', 'studio', 'beach', 'scan', 'already', 'hotel', 'smell', 'revelation', 'exposure', 'fleet', 'weather', 'damage', 'late', 'window', 'destruction', 'bulb', 'tropical', 'globe', 'their', 'living', 'deck', 'darkness', 'display', 'arrival', 'roof', 'financial', 'tourism', 'original', 'outfit', 'ocean', 'below', 'sunlight', 'bubble', 'storm', 'weaken', 'mirror', 'powerful', 'likelihood', 'building', 'arena', 'perceive', 'birthday', 'glass', 'descend', 'diminish', 'universe', 'crystal', 'porch', 'lion', 'emerging', 'shell', 'vs', 'attack', 'mall', 'earth', 'portrait', 'basement', 'consumer', 'nest', 'crew', 'fate', 'air', 'screen', 'assess', 'apple', 'cell', 'grandparent', 'convention', 'picture', 'palace', 'shark', 'operating', 'organism', 'electronic', 'elsewhere', 'earnings', 'estate', 'well-being', 'warehouse', 'ah', 'annual', 'origin', 'since', 'lobby', 'hurricane', 'immune', 'bedroom', 'retailer', 'plunge', 'drawer', 'holiday', 'landscape', 'merc

In [14]:
compare_vocabulary_hashes(EMBEDDING_MODEL, HierarchicalLsh(num_levels=10), search_terms)
# note that this takes 60 seconds to run

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with HierarchicalLsh (num_levels=10)
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Using 850 samples for hierarchical clustering
Maximum HierarchicalClustering hash length: 10
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 1110011100: ['hardware', 'apple', 'computer', 'technology', 'software']
Words for word orange, hash 101000110: ['green', 'orange', 'yellow', 'blue', 'purple', 'jacket', 'pink', 'shirt', 'golden', 'red']
Words for word banana, hash 1001110111: ['vegetable', 'bean', 'nut', 'coffee', 'fruit', 'peanut', 'banana']
Words for word grape, hash 1001110110: ['grape', 'wine']
Words for word fruit, hash 1001110111: ['vegetable', 'bean', 'nut', 'coffee', 'fruit', 'peanut', 'banana']
Words for word cheese, hash 1001110010: ['cheese', 'cream', 'steak', 'butter']
Words for word plane, hash 101110111: ['jet', 'flying', 'helicopter', 'crew', 'vessel', 'f

In [15]:
compare_vocabulary_hashes(EMBEDDING_MODEL, HierarchicalHyperplaneLsh(num_levels=10), search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model with HierarchicalHyperplaneLsh(num_levels=10)
Embedding vocabulary of 4319 words
Training hasher with all 4253 vocabulary embeddings
Using 4253 samples for hierarchical clustering
Building hierarchy over 4253 data points
Maximum HierarchicalClustering hash length: 13
Enriching clusters with parents and levels and hyperplanes
Hierarchical hyperplanes calculated
Hashing all 4253 vocabulary embeddings
Words for word apple, hash 00100100: ['apple', 'grape', 'fruit', 'liver']
Words for word orange, hash 01000110: ['orange']
Words for word banana, hash 001110001: ['salmon', 'carrot', 'nut', 'raw', 'potato', 'mushroom', 'pound', 'serving', 'fat', 'serve', 'peanut', 'banana']
Words for word grape, hash 00100100: ['apple', 'grape', 'fruit', 'liver']
Words for word fruit, hash 00100100: ['apple', 'grape', 'fruit', 'liver']
Words for word cheese, hash 001111010: ['cheese', 'chocolate', 'cream']
Words for word plane, hash 110

# Gensim similarity comparison

In [20]:
def print_gensim_word_neighbourhood(word, embeddings_model_wrapper):
    neighbourhood = []
    for neighbourhood_word, _ in embeddings_model_wrapper.embeddings_model.most_similar(word, topn=20):
        if neighbourhood_word in vocabulary:
            neighbourhood.append(neighbourhood_word)
    print(f'Gensim words for word {word}: {neighbourhood}')

def compare_gensim_word_neighbourhood(embeddings_model_name, search_terms):
    embeddings_model_wrapper = EmbeddingsModel.get(embeddings_model_name)
    print(f"Comparing vocabulary hashes for {embeddings_model_name} embeddings model")
    for term in search_terms:
        print_gensim_word_neighbourhood(term, embeddings_model_wrapper)

In [21]:
compare_gensim_word_neighbourhood(EMBEDDING_MODEL, search_terms)

Comparing vocabulary hashes for glove-wiki-gigaword-100 embeddings model
Gensim words for word apple: ['software', 'computer']
Gensim words for word orange: ['yellow', 'red', 'blue', 'green', 'pink', 'purple', 'black', 'lemon', 'brown', 'juice', 'white', 'gray', 'bright', 'cream', 'dark']
Gensim words for word banana: ['potato', 'fruit', 'peanut', 'sugar', 'nut', 'tomato', 'coffee', 'bean']
Gensim words for word grape: ['wine', 'fruit', 'tomato']
Gensim words for word fruit: ['vegetable', 'coffee', 'juice', 'flavor', 'banana', 'grape', 'tomato', 'tea']
Gensim words for word cheese: ['cream', 'butter', 'chocolate', 'bread', 'sandwich', 'sauce', 'pasta', 'potato', 'tomato', 'goat']
Gensim words for word plane: ['airplane', 'flight', 'jet', 'crash', 'helicopter', 'flying', 'aircraft', 'pilot', 'landing', 'crew', 'cargo', 'fly']
Gensim words for word train: ['bus', 'rail', 'passenger', 'traffic', 'truck', 'ride', 'car', 'plane', 'route']
Gensim words for word universe: ['planet', 'earth', 